In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval

In [2]:
metadata = pd.read_csv('movies_metadata.csv', low_memory = False)
# metadata.head().T

In [3]:
metadata.drop(metadata.index[metadata['id'] == '1997-08-20'], inplace = True)

In [4]:
metadata.drop(metadata.index[metadata['id'] == '2012-09-29'], inplace = True)

In [5]:
metadata.drop(metadata.index[metadata['id'] == '2014-01-01'], inplace = True)

In [6]:
metadata['id'] = metadata['id'].astype(str).astype(int)

In [7]:
cred = pd.read_csv('credits.csv')
# cred.head().T

In [8]:
metadata= metadata.merge(cred,on='id')
metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [9]:
metadata = metadata.drop(['belongs_to_collection', 'homepage', 'poster_path'], axis=1)

In [10]:
metadata['budget'] = metadata['budget'].astype(str).astype(float)

In [11]:
metadata['genres'] = metadata['genres'].apply(literal_eval)

In [12]:
gen = []

for i in metadata['genres']:
    l = []
    
    for j in i:
        l.append(j['name'])
#         print(j['name'])
    gen.append(l)

In [13]:
metadata['genres'] = pd.Series(gen)

In [14]:
metadata['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45533                 [Drama, Family]
45534                         [Drama]
45535       [Action, Drama, Thriller]
45536                              []
45537                              []
Name: genres, Length: 45538, dtype: object

In [15]:
metadata['cast'] = metadata['cast'].apply(literal_eval)

In [16]:
actors = []

for i in metadata['cast']:
    l = []
    for j in i:
        l.append(j['name'])
    
    actors.append(l)

In [17]:
metadata['actors'] = pd.Series(actors)

In [18]:
metadata['actors']

0        [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...
1        [Robin Williams, Jonathan Hyde, Kirsten Dunst,...
2        [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...
3        [Whitney Houston, Angela Bassett, Loretta Devi...
4        [Steve Martin, Diane Keaton, Martin Short, Kim...
                               ...                        
45533          [Leila Hatami, Kourosh Tahami, Elham Korda]
45534    [Angel Aquino, Perry Dizon, Hazel Orencio, Joe...
45535    [Erika Eleniak, Adam Baldwin, Julie du Page, J...
45536    [Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...
45537                                                   []
Name: actors, Length: 45538, dtype: object

In [19]:
metadata['crew']= metadata['crew'].apply(literal_eval)

In [20]:
metadata['crew'][0]

[{'credit_id': '52fe4284c3a36847f8024f49',
  'department': 'Directing',
  'gender': 2,
  'id': 7879,
  'job': 'Director',
  'name': 'John Lasseter',
  'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f4f',
  'department': 'Writing',
  'gender': 2,
  'id': 12891,
  'job': 'Screenplay',
  'name': 'Joss Whedon',
  'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f55',
  'department': 'Writing',
  'gender': 2,
  'id': 7,
  'job': 'Screenplay',
  'name': 'Andrew Stanton',
  'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f5b',
  'department': 'Writing',
  'gender': 2,
  'id': 12892,
  'job': 'Screenplay',
  'name': 'Joel Cohen',
  'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f61',
  'department': 'Writing',
  'gender': 0,
  'id': 12893,
  'job': 'Screenplay',
  'name': 'Alec Sokolow',
  'profile_path': '/v79vlRYi94BZUQnkkyzn

In [21]:
director = []
c = 0

for i in metadata['crew']:
    l = []
    for x in i:
        if x['job'] =='Director':
            l.append(x['name'])
#     print(l)
    director.append(l)


In [22]:
metadata['directors'] = pd.Series(director)

In [23]:
df = metadata.copy()

In [24]:
C = df['vote_average'].mean()
C

5.61823212913114

In [25]:
M = df['vote_count'].quantile(0.9)
M

160.0

In [26]:
def weightedRating(x, m =M, c=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return (v/(v+m) * R) + (m/(m+v) * C)

In [27]:
df['rating_score'] = df.apply(weightedRating, axis = 1)

In [28]:
df['production_companies'][1]

"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]"

In [29]:
df['production_companies'].replace('[]', "[{'name': 'Unknown'}]", inplace = True )

In [30]:
d = 0
for i in df['production_companies']:
    if i == '[]':
        d+=1
#         print ("[{'name': 'Unknown'}]")
#     else:
#         print(i)

print(d)

0


In [35]:
x = df['production_companies']

In [37]:
for i in x:
    if i == '[]':
        print('list')
        
    if i == '':
        print('nan')
        
    if i == np.nan:
        print('life sucks')

In [32]:
df['production_companies'][100]

"[{'name': 'Touchstone Pictures', 'id': 9195}]"

In [46]:
def changeValues(X):
    for i in X['production_companies']:
        return eval(X, {})
#     for i in X['production_companies']:
#         if i == '[]':
#             return "[{'name': 'Unknown'}]"
#         else:
#             return i

In [47]:
df['companies'] = df.apply(changeValues, axis=1)

TypeError: eval() arg 1 must be a string, bytes or code object

In [43]:
df['production_companies'] = df.apply(changeValues)

TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:
df['production_companies'][1000]

In [ ]:
df.columns

In [ ]:
companies = []

for i in df['production_companies']:
    l = []
    
    for j in i:
        print(j)
#         l.append(j['name'])